# Chapter: 15
## Section: Using dowhy for causal effect prediction

In [19]:
!pip install dowhy==0.5.1

In [20]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
import dowhy
from dowhy import CausalModel

In [21]:
# Loading scikit-leanr's diabates dataset
breast_cancer = load_breast_cancer()
data = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
data['target'] = breast_cancer.target

In [22]:
# Converting continuous mean radius feature to binary with the mean value as the trheshold
data['mean radius'] = data['mean radius'].gt(data['mean radius'].values.mean()).astype(int)
data=data.astype({'mean radius':'bool'}, copy=False)

In [23]:
common_causes_list = data.columns.values.tolist()
common_causes_list.remove('mean radius')
common_causes_list.remove('target')

In [24]:
model = CausalModel(
    data=data,
    treatment='mean radius',
    outcome='target',
    common_causes=common_causes_list
)

In [25]:
# Identifying causal effect
identified_est = model.identify_effect()

WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] y


In [ ]:
estimate = model.estimate_effect(identified_est,
                                 method_name='backdoor.propensity_score_matching')

In [27]:
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

## Realized estimand
b: target~mean radius+mean concave points+mean area+area error+concave points error+worst radius+mean symmetry+mean texture+compactness error+mean compactness+perimeter error+symmetry error+worst concavity+worst symmetry+worst compactness+worst concave points+worst texture+worst area+worst perimeter+mean fractal dimension+mean concavity+smoothness error+mean perimeter+worst fractal dimension+concavity error+worst smoothness+mean smoothness+radius error+texture error+fractal dimension error
Target units: ate

## Estimate
Mean value: -0.27065026362038663



In [ ]:
# Refute an estimate by replacing treatment with a randomly-generated placebo variable.
refute_results = model.refute_estimate(identified_est, estimate,
                                       method_name='placebo_treatment_refuter',
                                       placebo_type='permute', num_simulations=40)

In [29]:
print(refute_results)

Refute: Use a Placebo Treatment
Estimated effect:-0.27065026362038663
New effect:-0.000219683655536028
p value:0.49762065737736705

